In [1]:
import os
import openai
import pandas as pd

In [2]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [5]:
# Create dataframe with the tabular data contained in the CSV which is the object of the lecture
df= pd.read_csv("sales_data_sample.csv")

In [6]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [10]:
# Sales by quarter using simple pandas function
# df.groupby("QTR_ID").sum()['SALES']

QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64

In [ ]:
# Let's use SQL, for that we'll create a temporary file in RAM with the dataframe
# The idea is to ask something like "What was the total sum of sales per quarter?" and get an anser from a SQL query


In [11]:
# creating SQL database with pandas
import sqlalchemy

In [12]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [13]:
# TEMP DB in RAM

# PUSH Pandas DF --> TEMP DB

# SQL query on TEMP DB

In [14]:
temp_db = create_engine('sqlite:///:memory:',echo=True)
# echo=True so we can see outputs

In [16]:
data = df.to_sql(name='Sales',con=temp_db)
# I see this output because echo=True
# con stand for connection

2023-03-04 19:46:03,570 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-03-04 19:46:03,571 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-04 19:46:03,573 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-03-04 19:46:03,573 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-04 19:46:03,576 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-04 19:46:03,578 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

In [19]:
#using sqlalchemy we can stablish a connection to this temporary database and query for a result
with temp_db.connect() as conn:
    # makes the connection
    # run code indentation/block
    result = conn.execute(text("SELECT SUM(SALES) from Sales"))
    # autoclose connection

2023-03-04 19:52:51,885 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) from Sales
2023-03-04 19:52:51,887 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ()


In [20]:
result.all()

[(10032628.85000001,)]

In [ ]:
# NLP to SQL
# WE'LL NEED SOME HANDLER FUNCTIONS NI ORDER TO GIVE gpt-3 THE INFORMATION IT NEEDS ABOUT TABLE STRUCTURE
# We'll create a few functions
# 1. Automaticall creating table structure definition from DF for OpenAI API
# 2. Grabbing user NLP input
# 3. Combining prompts for an OpenAI API call

In [ ]:
### sqlite SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months
# SELECT

In [23]:
# function that creates that intro prompt given a df
# this is a common task and OpenAI has some suggestions in its own documentation, for example starting it as a comment with ### so it completes
def create_table_definition(df):
    prompt = """### sqlite SQL table, with its properties
    #
    # Sales({}) 
    #
    """.format(",".join(str(col) for col in df.columns))
    
    return prompt

In [24]:
df.columns

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'MONTH_ID', 'YEAR_ID',
       'PRODUCTLINE', 'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE',
       'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE',
       'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEALSIZE'],
      dtype='object')

In [28]:
",".join(str(col) for col in df.columns)

'ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE'

In [30]:
print(create_table_definition(df))

### sqlite SQL table, with its properties
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE) 
    #
    


In [31]:
def prompt_input():
    nlp_text = input('Enter the info you want')
    return nlp_text

In [32]:
prompt_input()

Enter the info you wantI want the sales made by the best department


'I want the sales made by the best department'

In [33]:
def combine_prompts(df,query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition+query_init_string
#This is the prompt engineering part
    

In [34]:
nlp_text = prompt_input() #NLP
combine_prompts(df,nlp_text) # DF + query that does... + NLP

Enter the info you wanttotal sales per quarter


'### sqlite SQL table, with its properties\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE) \n    #\n    ### A query to answer: total sales per quarter\nSELECT'

In [ ]:
# ^note the point of hardcoding "SELECT" in the output is understood when we think of GPT as a text completion engine
# by orcing "SELECT" into the answer, probabillistically GPT is more likely to complete a SQL query

In [36]:
response = openai.Completion.create(
        model='code-davinci-002',
        prompt = combine_prompts(df,nlp_text),
        temperature = 0,
        max_tokens = 150,
        top_p = 1.0,
        frequency_penalty=0,
        presence_penalty=0,
        stop= ['#',';']
)

In [40]:
response['choices'][0]['text']

' QTR_ID, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY QTR_ID\nORDER BY QTR_ID'

In [47]:
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(' '):
        query = "SELECT"+query
    return query
        

In [48]:
handle_response(response)

'SELECT QTR_ID, SUM(SALES) AS TOTAL_SALES\nFROM Sales\nGROUP BY QTR_ID\nORDER BY QTR_ID'

In [43]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-03-04 22:25:39,982 INFO sqlalchemy.engine.Engine SELECT QTR_ID, SUM(SALES) AS TOTAL_SALES
FROM Sales
GROUP BY QTR_ID
ORDER BY QTR_ID
2023-03-04 22:25:39,984 INFO sqlalchemy.engine.Engine [generated in 0.00223s] ()


In [46]:
result.all()

[(1, 2350817.7300000004),
 (2, 2048120.2999999986),
 (3, 1758910.8099999994),
 (4, 3874780.01)]

In [49]:
nlp_text = prompt_input() #NLP
combine_prompts(df,nlp_text) # DF + query that does... + NLP

Enter the info you wantnumber of cities per state


'### sqlite SQL table, with its properties\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE) \n    #\n    ### A query to answer: number of cities per state\nSELECT'

In [50]:
response = openai.Completion.create(
        model='code-davinci-002',
        prompt = combine_prompts(df,nlp_text),
        temperature = 0,
        max_tokens = 150,
        top_p = 1.0,
        frequency_penalty=0,
        presence_penalty=0,
        stop= ['#',';']
)

In [51]:
handle_response(response)

'SELECT COUNT(DISTINCT CITY) AS COUNT_CITY, STATE FROM Sales GROUP BY STATE ORDER BY COUNT_CITY DESC'

In [53]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-03-04 22:41:04,251 INFO sqlalchemy.engine.Engine SELECT COUNT(DISTINCT CITY) AS COUNT_CITY, STATE FROM Sales GROUP BY STATE ORDER BY COUNT_CITY DESC
2023-03-04 22:41:04,252 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ()


In [55]:
result.all()

[(39, None),
 (10, 'CA'),
 (4, 'MA'),
 (3, 'CT'),
 (2, 'Victoria'),
 (2, 'PA'),
 (2, 'NY'),
 (2, 'NSW'),
 (2, 'BC'),
 (1, 'Tokyo'),
 (1, 'Queensland'),
 (1, 'Quebec'),
 (1, 'Osaka'),
 (1, 'NV'),
 (1, 'NJ'),
 (1, 'NH'),
 (1, 'Isle of Wight')]